In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import sagemaker
from datetime import datetime

# Read dataset

In [3]:
DATA_PATH = 'data/'
df = pd.read_csv('sagemaker-autopilot/data/200514COVID19MEXICO2.csv', encoding = "ISO-8859-1", sep=',')
df.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-05-14,03210d,2,4,31,1,31,31,50,1,...,2,1,2,2,99,1,99,MÃ©xico,99,97
1,2020-05-14,0e00d2,2,4,9,1,9,9,16,1,...,2,2,2,2,99,1,99,MÃ©xico,99,97
2,2020-05-14,03f822,2,4,9,2,12,23,8,1,...,2,2,2,2,99,1,99,MÃ©xico,99,97
3,2020-05-14,002430,2,4,22,2,22,22,14,1,...,2,2,2,2,99,1,99,MÃ©xico,99,97
4,2020-05-14,0930ff,2,4,9,2,9,15,121,2,...,1,2,2,2,99,1,99,MÃ©xico,99,2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155932 entries, 0 to 155931
Data columns (total 35 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   FECHA_ACTUALIZACION  155932 non-null  object
 1   ID_REGISTRO          155932 non-null  object
 2   ORIGEN               155932 non-null  int64 
 3   SECTOR               155932 non-null  int64 
 4   ENTIDAD_UM           155932 non-null  int64 
 5   SEXO                 155932 non-null  int64 
 6   ENTIDAD_NAC          155932 non-null  int64 
 7   ENTIDAD_RES          155932 non-null  int64 
 8   MUNICIPIO_RES        155932 non-null  int64 
 9   TIPO_PACIENTE        155932 non-null  int64 
 10  FECHA_INGRESO        155932 non-null  object
 11  FECHA_SINTOMAS       155932 non-null  object
 12  FECHA_DEF            155932 non-null  object
 13  INTUBADO             155932 non-null  int64 
 14  NEUMONIA             155932 non-null  int64 
 15  EDAD                 155932 non-nu

In [5]:
df.describe()

,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,INTUBADO,NEUMONIA,...,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,UCI
count,155932.000000,155932.000000,155932.000000,155932.000000,155932.000000,155932.000000,155932.000000,155932.000000,155932.000000,155932.000000,...,155932.000000,155932.000000,155932.000000,155932.000000,155932.000000,155932.000000,155932.000000,155932.00000,155932.000000,155932.000000
mean,1.635117,9.619289,14.545982,1.510415,15.992362,14.819152,35.349338,1.255785,72.699221,1.827232,...,2.210213,2.481351,2.361626,2.223694,2.352455,2.276698,34.330112,1.89836,98.565945,72.698413
std,0.481399,8.005199,7.930656,0.499893,11.228051,7.849520,44.719936,0.436303,41.472949,0.831616,...,6.068848,7.070229,6.108154,6.111388,5.995977,5.971359,46.087682,0.65906,6.487187,41.475071
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000
25%,1.000000,4.000000,9.000000,1.000000,9.000000,9.000000,7.000000,1.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.00000,99.000000,2.000000
50%,2.000000,12.000000,14.000000,2.000000,15.000000,14.000000,20.000000,1.000000,97.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000,99.000000,97.000000
75%,2.000000,12.000000,19.000000,2.000000,21.000000,19.000000,46.000000,2.000000,97.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,99.000000,2.00000,99.000000,97.000000
max,2.000000,99.000000,32.000000,2.000000,99.000000,32.000000,999.000000,2.000000,99.000000,99.000000,...,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,99.000000,3.00000,99.000000,99.000000


enc = OrdinalEncoder()

enc.fit(df)

data = enc.transform(df)

In [ ]:
# Split dataset
enc = OrdinalEncoder()

enc

enc.fit(df)

data = enc.transform(df)
print(data)

In [6]:
def calc_days (row):
    dateinitial = row["FECHA_SINTOMAS"]
    datefinal = row["FECHA_INGRESO"]

    datetime_initial = datetime.strptime(dateinitial, '%Y-%m-%d').date()
    datetime_final = datetime.strptime(datefinal, '%Y-%m-%d').date()
    diff = datetime_final - datetime_initial
    days = diff.total_seconds()/86400
    return days;

In [7]:
df['days'] = df.apply (lambda row: calc_days(row), axis=1)
df.describe()

,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,INTUBADO,NEUMONIA,...,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,UCI,days
count,155932.000000,155932.000000,155932.000000,155932.000000,155932.000000,155932.000000,155932.000000,155932.000000,155932.000000,155932.000000,...,155932.000000,155932.000000,155932.000000,155932.000000,155932.000000,155932.000000,155932.00000,155932.000000,155932.000000,155932.000000
mean,1.635117,9.619289,14.545982,1.510415,15.992362,14.819152,35.349338,1.255785,72.699221,1.827232,...,2.481351,2.361626,2.223694,2.352455,2.276698,34.330112,1.89836,98.565945,72.698413,3.539344
std,0.481399,8.005199,7.930656,0.499893,11.228051,7.849520,44.719936,0.436303,41.472949,0.831616,...,7.070229,6.108154,6.111388,5.995977,5.971359,46.087682,0.65906,6.487187,41.475071,3.230464
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,0.000000
25%,1.000000,4.000000,9.000000,1.000000,9.000000,9.000000,7.000000,1.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.00000,99.000000,2.000000,1.000000
50%,2.000000,12.000000,14.000000,2.000000,15.000000,14.000000,20.000000,1.000000,97.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000,99.000000,97.000000,3.000000
75%,2.000000,12.000000,19.000000,2.000000,21.000000,19.000000,46.000000,2.000000,97.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,99.000000,2.00000,99.000000,97.000000,5.000000
max,2.000000,99.000000,32.000000,2.000000,99.000000,32.000000,999.000000,2.000000,99.000000,99.000000,...,98.000000,98.000000,98.000000,98.000000,98.000000,99.000000,3.00000,99.000000,99.000000,90.000000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155932 entries, 0 to 155931
Data columns (total 36 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   FECHA_ACTUALIZACION  155932 non-null  object 
 1   ID_REGISTRO          155932 non-null  object 
 2   ORIGEN               155932 non-null  int64  
 3   SECTOR               155932 non-null  int64  
 4   ENTIDAD_UM           155932 non-null  int64  
 5   SEXO                 155932 non-null  int64  
 6   ENTIDAD_NAC          155932 non-null  int64  
 7   ENTIDAD_RES          155932 non-null  int64  
 8   MUNICIPIO_RES        155932 non-null  int64  
 9   TIPO_PACIENTE        155932 non-null  int64  
 10  FECHA_INGRESO        155932 non-null  object 
 11  FECHA_SINTOMAS       155932 non-null  object 
 12  FECHA_DEF            155932 non-null  object 
 13  INTUBADO             155932 non-null  int64  
 14  NEUMONIA             155932 non-null  int64  
 15  EDAD             

In [10]:
df = df.drop(['FECHA_INGRESO'], axis=1)
df = df.drop(['FECHA_SINTOMAS'], axis=1)
df = df.drop(['FECHA_ACTUALIZACION'], axis=1)
df = df.drop(['FECHA_DEF'], axis=1)
df = df.drop(['PAIS_ORIGEN'], axis=1)
df = df.drop(['PAIS_NACIONALIDAD'], axis=1)
df = df.drop(['ID_REGISTRO'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155932 entries, 0 to 155931
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ORIGEN              155932 non-null  int64  
 1   SECTOR              155932 non-null  int64  
 2   ENTIDAD_UM          155932 non-null  int64  
 3   SEXO                155932 non-null  int64  
 4   ENTIDAD_NAC         155932 non-null  int64  
 5   ENTIDAD_RES         155932 non-null  int64  
 6   MUNICIPIO_RES       155932 non-null  int64  
 7   TIPO_PACIENTE       155932 non-null  int64  
 8   INTUBADO            155932 non-null  int64  
 9   NEUMONIA            155932 non-null  int64  
 10  EDAD                155932 non-null  int64  
 11  NACIONALIDAD        155932 non-null  int64  
 12  EMBARAZO            155932 non-null  int64  
 13  HABLA_LENGUA_INDIG  155932 non-null  int64  
 14  DIABETES            155932 non-null  int64  
 15  EPOC                155932 non-nul

In [11]:
datos = df.drop(df[df.INTUBADO > 2].index)
datos.describe()

,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,INTUBADO,NEUMONIA,...,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,UCI,days
count,39855.000000,39855.000000,39855.000000,39855.000000,39855.000000,39855.000000,39855.000000,39855.0,39855.000000,39855.000000,...,39855.000000,39855.000000,39855.000000,39855.000000,39855.000000,39855.000000,39855.000000,39855.000000,39855.000000,39855.000000
mean,1.446167,8.385196,14.951298,1.600176,16.318655,15.250508,39.652490,2.0,1.922118,1.396713,...,3.030862,2.705507,2.591469,2.647874,2.601229,50.900565,1.726935,98.709371,1.918956,3.604767
std,0.497100,8.978108,7.878701,0.489868,9.944026,7.779665,50.194692,0.0,0.267990,0.489222,...,10.220181,8.536080,8.727205,8.237715,8.187200,48.621548,0.707217,5.310891,0.559462,3.456053
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.0,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,1.000000,4.000000,9.000000,1.000000,9.000000,9.000000,7.000000,2.0,2.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,99.000000,2.000000,1.000000
50%,1.000000,6.000000,15.000000,2.000000,15.000000,15.000000,21.000000,2.0,2.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,99.000000,2.000000,99.000000,2.000000,3.000000
75%,2.000000,12.000000,21.000000,2.000000,21.000000,21.000000,57.000000,2.0,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,99.000000,2.000000,99.000000,2.000000,6.000000
max,2.000000,99.000000,32.000000,2.000000,99.000000,32.000000,570.000000,2.0,2.000000,2.000000,...,98.000000,98.000000,98.000000,98.000000,98.000000,99.000000,3.000000,99.000000,99.000000,90.000000


In [13]:
# Shuffle df
shuffled_df = datos.sample(frac=1,random_state=4)

# Put all the fraud class in a separate dataset.
CHD_df = shuffled_df.loc[shuffled_df['INTUBADO'] == 1]

#Randomly select 492 observations from the non-fraud (majority class)
non_CHD_df = shuffled_df.loc[shuffled_df['INTUBADO'] == 2].sample(n=3000,random_state=42)

# Concatenate both dataframes again
normalized_df = pd.concat([CHD_df, non_CHD_df])

# check new class counts
normalized_df.INTUBADO.value_counts()

1    3104
2    3000
Name: INTUBADO, dtype: int64

In [14]:
train, test = train_test_split(normalized_df, test_size=0.2)

In [16]:
print('all:  ', len(normalized_df))
print('train:', len(train))
print('test: ', len(test))

train.describe()

all:   6104
train: 4883
test:  1221


,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,INTUBADO,NEUMONIA,...,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,UCI,days
count,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,4883.0,4883.000000,4883.000000,...,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000
mean,1.428835,9.467131,14.700389,1.631374,16.338931,15.091747,40.191071,2.0,1.492525,1.249642,...,3.915830,3.513619,3.293467,3.373746,3.269302,35.088470,1.664550,98.561745,1.710834,3.959042
std,0.494960,6.975075,7.656769,0.482482,10.431769,7.543197,53.856081,0.0,0.499995,0.432850,...,13.677044,12.197859,11.995992,11.661034,11.429700,46.145642,0.708197,6.515197,0.453422,3.372838
min,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.0,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,1.000000,4.000000,9.000000,1.000000,9.000000,9.000000,7.000000,2.0,1.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,99.000000,1.000000,1.000000
50%,1.000000,12.000000,14.000000,2.000000,15.000000,15.000000,20.000000,2.0,1.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,99.000000,2.000000,3.000000
75%,2.000000,12.000000,20.000000,2.000000,21.000000,20.000000,55.000000,2.0,2.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,99.000000,2.000000,99.000000,2.000000,6.000000
max,2.000000,99.000000,32.000000,2.000000,99.000000,32.000000,565.000000,2.0,2.000000,2.000000,...,98.000000,98.000000,98.000000,98.000000,98.000000,99.000000,3.000000,99.000000,2.000000,33.000000


In [18]:
train_file = 'sagemaker-autopilot/data/covid-train6.csv'
pd.DataFrame.from_records(train).to_csv(train_file, index=False, header=True, sep=',')

test_file = 'sagemaker-autopilot/data/covid-test6.csv'
pd.DataFrame.from_records(test).to_csv(test_file, index=False, header=True, sep=',')

In [19]:
session = sagemaker.Session()
uri = session.upload_data(path=train_file, key_prefix='covid')
print(uri)

s3://sagemaker-us-west-2-651677422791/covid/covid-train6.csv
